# Strands Agents와 AgentCore Memory (장기 메모리)

## 소개

이 튜토리얼은 훅을 통해 AgentCore Memory와 통합된 Strands agents를 사용하여 **지능형 고객 지원 에이전트**를 구축하는 방법을 보여줍니다. 고객 상호작용 기록, 구매 세부사항 기억, 이전 대화와 사용자 선호도에 기반한 개인화된 지원 제공을 위한 장기 메모리에 초점을 맞춥니다.

### 튜토리얼 세부사항

| 정보                | 세부사항                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형        | 장기 대화형                                                                         |
| 에이전트 유형        | 고객 지원                                                                         |
| 에이전트 프레임워크   | Strands Agents                                                                   |
| LLM 모델            | Anthropic Claude Sonnet 3.7                                                      |
| 튜토리얼 구성요소     | AgentCore 시맨틱 및 사용자 선호도 메모리 추출, 메모리 저장 및 검색을 위한 훅              |
| 예제 복잡도          | 중급                                                                             |

배울 내용:
- 장기 전략으로 AgentCore Memory 설정
- 자동 저장 및 검색을 위한 메모리 훅 생성
- 지속적인 메모리를 가진 고객 지원 에이전트 구축
- 이전 상호작용의 컨텍스트로 고객 문제 처리

### 시나리오 컨텍스트
이 예제에서는 **고객 지원 사용 사례**를 구축합니다. 에이전트는 주문 기록, 선호도, 이전 문제를 포함한 고객 컨텍스트를 기억하여 더 개인화되고 효과적인 지원을 가능하게 합니다. 고객과의 대화는 메모리 훅을 사용하여 자동으로 저장되어 중요한 세부사항이 절대 손실되지 않도록 합니다. 시맨틱 및 사용자 선호도와 같은 여러 메모리 전략을 사용하여 에이전트는 광범위한 관련 정보를 캐처할 수 있습니다. 이 설정은 에이전트가 고객의 기록과 선호도를 완전히 인식하면서 문제를 해결할 수 있게 합니다. 또한 에이전트는 웹 검색 기능과 통합되어 필요에 따라 최신 제품 정보와 문제 해결 지침을 쉽게 제공할 수 있습니다.

## 아키텍처

<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>


## 전제조건

이 튜토리얼을 실행하려면 다음이 필요합니다:
- Python 3.10+
- Amazon Bedrock AgentCore Memory 권한이 있는 AWS 자격 증명
- Amazon Bedrock AgentCore SDK

## 1단계: 종속성 설치 및 설정
이 노트북이 작동하도록 필요한 모든 라이브러리를 가져오고 클라이언트를 정의하는 것부터 시작해보겠습니다.

In [ ]:
!pip install -qr requirements.txt

In [ ]:
import logging
import json
from typing import Dict
from datetime import datetime
from botocore.exceptions import ClientError

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger("customer-support")

# Import required modules
from strands import Agent, tool
from strands.hooks import AfterInvocationEvent, HookProvider, HookRegistry, MessageAddedEvent
from ddgs import DDGS

In [ ]:
# Configuration - Replace with the correct values
REGION = "us-west-2"
CUSTOMER_ID = "customer_001"
SESSION_ID = f"support_{datetime.now().strftime('%Y%m%d%H%M%S')}"

## 2단계: 고객 지원을 위한 메모리 리소스 생성

고객 지원을 위해 여러 메모리 전략을 사용합니다:
- **USER_PREFERENCE**: 고객 선호도와 행동 캐처
- **SEMANTIC**: 주문 사실과 제품 정보 저장

In [ ]:
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.constants import StrategyType

# Initialize Memory Client
client = MemoryClient(region_name=REGION)
memory_name = "CustomerSupportMemory"

# Define memory strategies for customer support
strategies = [
    {
        StrategyType.USER_PREFERENCE.value: {
            "name": "CustomerPreferences",
            "description": "Captures customer preferences and behavior",
            "namespaces": ["support/customer/{actorId}/preferences"]
        }
    },
    {
        StrategyType.SEMANTIC.value: {
            "name": "CustomerSupportSemantic",
            "description": "Stores facts from conversations",
            "namespaces": ["support/customer/{actorId}/semantic"],
        }
    }
]

# Create memory resource
try:
    memory = client.create_memory_and_wait(
        name=memory_name,
        strategies=strategies,         # Define the memory strategies
        description="Memory for customer support agent",
        event_expiry_days=90,          # Memories expire after 90 days
    )
    memory_id = memory['id']
    logger.info(f"✅ Created memory: {memory_id}")
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # If memory already exists, retrieve its ID
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"Memory already exists. Using existing memory ID: {memory_id}")
except Exception as e:
    # Handle any errors during memory creation
    logger.info(f"❌ ERROR: {e}")
    import traceback
    traceback.print_exc()
    # Cleanup on error - delete the memory if it was partially created
    if memory_id:
        try:
            client.delete_memory_and_wait(memoryId=memory_id,max_wait = 300)
            logger.info(f"Cleaned up memory: {memory_id}")
        except Exception as cleanup_error:
            logger.info(f"Failed to clean up memory: {cleanup_error}")

할당한 전략이 메모리에 포함되어 있는지 확인해보겠습니다

In [ ]:
strategies = client.get_memory_strategies(memory_id)
print(json.dumps(strategies, indent=2, default=str))

## 3단계: 에이전트 도구 생성

In [ ]:
from ddgs.exceptions import DDGSException, RatelimitException
from ddgs import DDGS

@tool
def web_search(query: str, max_results: int = 3) -> str:
    """Search the web for product information, troubleshooting guides, or support articles.
    
    Args:
        query: Search query for product info or troubleshooting
        max_results: Maximum number of results to return
    
    Returns:
        Search results with titles and snippets
    """
    try:
        results = DDGS().text(query, region="us-en", max_results=max_results)
        if not results:
            return "No search results found."
        
        formatted_results = []
        for i, result in enumerate(results, 1):
            formatted_results.append(f"{i}. {result.get('title', 'No title')}\n   {result.get('body', 'No description')}")
        
        return "\n".join(formatted_results)
    except RatelimitException:
        return "Rate limit reached: Please try again after a short delay."
    except DuckDuckGoSearchException as d:
        return f"Search Error: {d}"
    except Exception as e:
        return f"Search error: {str(e)}"

logger.info("✅ Web search tool ready")

@tool
def check_order_status(order_number: str) -> str:
    """Check the status of a customer order.
    
    Args:
        order_number: The order number to check
    
    Returns:
        Order status information
    """
    # Simulate order lookup
    mock_orders = {
        "123456": "iPhone 15 Pro - Delivered on June 5, 2025",
        "654321": "Sennheiser Headphones - Delivered on June 25, 2025, 1-year warranty active",
        "789012": "Samsung Galaxy S23 - In transit, expected delivery on July 1, 2025",
    }
    
    return mock_orders.get(order_number, f"Order {order_number} not found. Please verify the order number.")

logger.info("✅ Check Order Status tool ready")

## 4단계: 고객 지원을 위한 메모리 훅 프로바이더 생성
훅은 에이전트의 실행 생명주기의 특정 지점에서 실행되는 특수 함수입니다. 우리의 사용자 정의 훅 프로바이더는 다음을 통해 고객 지원 컨텍스트를 자동으로 관리합니다:
- **지원 상호작용 저장** 각 응답 후
- **관련 컨텍스트 검색 및 주입** 새로운 쿼리 처리 시 이전 주문과 선호도에서.

이는 수동 관리 없이 원활한 메모리 경험을 만들어줍니다

In [ ]:
# Helper function to get namespaces from memory strategies list
def get_namespaces(mem_client: MemoryClient, memory_id: str) -> Dict:
    """Get namespace mapping for memory strategies."""
    strategies = mem_client.get_memory_strategies(memory_id)
    return {i["type"]: i["namespaces"][0] for i in strategies}

In [ ]:
class CustomerSupportMemoryHooks(HookProvider):
    """Memory hooks for customer support agent"""
    
    def __init__(self, memory_id: str, client: MemoryClient, actor_id: str, session_id: str):
        self.memory_id = memory_id
        self.client = client
        self.actor_id = actor_id
        self.session_id = session_id
        self.namespaces = get_namespaces(self.client, self.memory_id)

    
    def retrieve_customer_context(self, event: MessageAddedEvent):
        """Retrieve customer context before processing support query"""
        messages = event.agent.messages
        if messages[-1]["role"] == "user" and "toolResult" not in messages[-1]["content"][0]:
            user_query = messages[-1]["content"][0]["text"]
            
            try:
                # Retrieve customer context from all namespaces
                all_context = []
                
                for context_type, namespace in self.namespaces.items():
                    memories = self.client.retrieve_memories(
                        memory_id=self.memory_id,
                        namespace=namespace.format(actorId=self.actor_id),
                        query=user_query,
                        top_k=3
                    )
                    
                    for memory in memories:
                        if isinstance(memory, dict):
                            content = memory.get('content', {})
                            if isinstance(content, dict):
                                text = content.get('text', '').strip()
                                if text:
                                    all_context.append(f"[{context_type.upper()}] {text}")
                
                # Inject customer context into the query
                if all_context:
                    context_text = "\n".join(all_context)
                    original_text = messages[-1]["content"][0]["text"]
                    messages[-1]["content"][0]["text"] = (
                        f"Customer Context:\n{context_text}\n\n{original_text}"
                    )
                    logger.info(f"Retrieved {len(all_context)} customer context items")
                    
            except Exception as e:
                logger.error(f"Failed to retrieve customer context: {e}")
    
    def save_support_interaction(self, event: AfterInvocationEvent):
        """Save support interaction after agent response"""
        try:
            messages = event.agent.messages
            if len(messages) >= 2 and messages[-1]["role"] == "assistant":
                # Get last customer query and agent response
                customer_query = None
                agent_response = None
                
                for msg in reversed(messages):
                    if msg["role"] == "assistant" and not agent_response:
                        agent_response = msg["content"][0]["text"]
                    elif msg["role"] == "user" and not customer_query and "toolResult" not in msg["content"][0]:
                        customer_query = msg["content"][0]["text"]
                        break
                
                if customer_query and agent_response:
                    # Save the support interaction
                    self.client.create_event(
                        memory_id=self.memory_id,
                        actor_id=self.actor_id,
                        session_id=self.session_id,
                        messages=[(customer_query, "USER"), (agent_response, "ASSISTANT")]
                    )
                    logger.info("Saved support interaction to memory")
                    
        except Exception as e:
            logger.error(f"Failed to save support interaction: {e}")
    
    def register_hooks(self, registry: HookRegistry) -> None:
        """Register customer support memory hooks"""
        registry.add_callback(MessageAddedEvent, self.retrieve_customer_context)
        registry.add_callback(AfterInvocationEvent, self.save_support_interaction)
        logger.info("Customer support memory hooks registered")

### 5단계: 고객 지원 에이전트 생성

In [ ]:
# Create memory hooks for customer support
support_hooks = CustomerSupportMemoryHooks(
    memory_id=memory_id,
    client=client,
    actor_id=CUSTOMER_ID,
    session_id=SESSION_ID
)

# Create customer support agent
support_agent = Agent(
    hooks=[support_hooks],
    model ="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    tools=[web_search, check_order_status],
    system_prompt="""You are a helpful customer support agent with access to customer history and order information. 
    
    Your role:
    - Help customers with their orders, returns, and product issues
    - Use customer context to provide personalized support
    - Search for product information when needed
    - Be empathetic and solution-focused
    - Reference previous orders and preferences when relevant
    
    Always be professional, helpful, and aim to resolve customer issues efficiently."""
)

print("✅ Customer support agent created with memory capabilities")

### 6단계: 고객 기록 시드

메모리 기능을 보여주기 위해 이전 고객 상호작용을 추가해보겠습니다.

In [ ]:
# Seed with previous customer interactions
previous_interactions = [
    ("I bought a new iPhone 15 Pro on June 1st, 2025. Order number is 123456.", "USER"),
    ("Thank you for your purchase! I can see your iPhone 15 Pro order #123456 was delivered successfully. How can I help you today?", "ASSISTANT"),
    ("I also ordered Sennheiser headphones on June 20th. Order number 654321. They came with 1-year warranty.", "USER"),
    ("Perfect! I have your Sennheiser headphones order #654321 on file with the 1-year warranty. Both your iPhone and headphones should work great together.", "ASSISTANT"),
    ("I'm looking for a good laptop. I prefer ThinkPad models.", "USER"),
    ("Great choice! ThinkPads are excellent for their durability and performance. Let me help you find the right model for your needs.", "ASSISTANT")
]

# Save previous interactions
try:
    client.create_event(
        memory_id=memory_id,
        actor_id=CUSTOMER_ID,
        session_id="previous_session",
        messages=previous_interactions
    )
    print("✅ Seeded customer history")
except Exception as e:
    print(f"⚠️ Error seeding history: {e}")

#### 에이전트가 준비되었습니다. 

### 고객 지원 시나리오를 테스트해보겠습니다

In [ ]:
# Test 1: Customer reports iPhone issue
response1 = support_agent("My iPhone is running very slow and gets hot when charging. Can you help?")
print(f"Support Agent: {response1}")

In [ ]:
# Test 2: Bluetooth connectivity issue
response2 = support_agent("My iPhone won't connect to my Sennheiser headphones via Bluetooth. How do I fix this?")
print(f"Support Agent: {response2}")

In [ ]:
# Test 3: Check order status
response3 = support_agent("Can you check the status of my recent orders?")
print(f"Support Agent: {response3}")

In [ ]:
# Test 4: Product recommendation based on preferences
response4 = support_agent("I'm still interested in buying a laptop. What ThinkPad models do you recommend?")
print(f"Support Agent: {response4}")

### 고객 메모리 저장 확인

In [ ]:
# Check stored customer memories
print("\n📚 Customer Memory Summary:")
print("=" * 50)

namespaces_dict = get_namespaces(client, memory_id)
for context_type, namespace_template in namespaces_dict.items():
    namespace = namespace_template.replace("{actorId}", CUSTOMER_ID)
    
    try:
        memories = client.retrieve_memories(
            memory_id=memory_id,
            namespace=namespace,
            query="customer orders and preferences",
            top_k=3
        )
        
        print(f"\n{context_type.upper()} ({len(memories)} items):")
        for i, memory in enumerate(memories, 1):
            if isinstance(memory, dict):
                content = memory.get('content', {})
                if isinstance(content, dict):
                    text = content.get('text', '')[:150] + "..."
                    print(f"  {i}. {text}")
                    
    except Exception as e:
        print(f"Error retrieving {context_type} memories: {e}")

print("\n" + "=" * 50)


#### 고객 지원 튜토리얼 완료! 🎉
주요 요점:
- 메모리 훅이 지원 세션 간 고객 컨텍스트를 자동으로 관리
- 다중 전략 메모리가 대화에서 주문, 선호도, 사실을 캐처
- 에이전트가 고객 기록에 기반한 개인화된 지원 제공 가능
- 주문 조회 및 웹 검색 기능을 위한 도구 통합 가능
- 지속적인 메모리로 고객 지원이 더 효율적이 됨

## 정리

### 선택사항: 메모리 리소스 삭제

In [ ]:
# Uncomment to delete the memory resource
# try:
#     client.delete_memory_and_wait(memory_id=memory_id)
#     print(f"✅ Deleted memory resource: {memory_id}")
# except Exception as e:
#     print(f"Error deleting memory: {e}")